In [ ]:
import datetime
import numpy as np
import pandas as pd
import requests
import random
import time
from bs4 import BeautifulSoup
from google.cloud import storage
import jieba as jb
import re

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # bucket_name = "your-bucket-name"
    # source_file_name = "local/path/to/file"
    # destination_blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        "File {} uploaded to {}.".format(
            source_file_name, destination_blob_name
        )
    )



def kerword(df):
    # 自訂停用詞
    myself_stopword = ['保險','保單','歲','險種','規劃','檢視','討論','請問']

    # 定義刪除除字母、漢字以外的所有符號的函數
    def remove_punctuation(line):
        line = str(line)
        if line.strip()=='':
            return ''
        rule = re.compile(u"[^a-zA-Z\u4E00-\u9FA5]")
        line = rule.sub('',line)
        return line 

    # 刪除停用词
    def delete_stopwords(df, myself_stopword):  
        stopwords = ['"', '#', '$', '&', '(', ')', '*', '+', ',', '-', '--', '.', '...', '......', '...................', './', '.一', '.数', '.日', '/', '//', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '://', '::', ';', '<', '=', '>', '?', '@', 'Lex', '[', ']', '_', 'exp', 'sub', 'sup', '}', '~~~~', '·', '×', '×××', 'Δ', 'Ψ', 'γ', 'μ', 'φ', 'φ．', 'В', '—', '——', '———', '‘', '’', '’‘', '“', '”', '”，', '……', '…………………………………………………③', '′∈', '′｜', '℃', 'Ⅲ', '↑', '→', '∈［', '∪φ∈', '≈', '①', '②', '②ｃ', '③', '③］', '④', '⑤', '⑥', '⑦', '⑧', '⑨', '⑩', '──', '■', '▲', '、', '。', '〉', '《', '》', '》），', '」', '『', '』', '【', '】', '〔', '〕', '〕〔', '㈧', '一', '一.', '一一', '一個', '一些', '一何', '一切', '一則', '一則通過', '一方面', '一旦', '一來', '一樣', '一番', '一直', '一般', '一轉眼', '萬一', '三天兩頭', '三番兩次', '三番五次', '上', '上下', '上去', '上來', '下', '不', '不下', '不了', '不亦樂乎', '不僅', '不僅僅', '不僅僅是', '不會', '不但', '不光', '不免', '不再', '不力', '不單', '不只', '不可開交', '不可抗拒', '不同', '不外', '不外乎', '不大', '不如', '不妨', '不定', '不對', '不少', '不盡', '不盡然', '不巧', '不已', '不常', '不得', '不得不', '不得了', '不得已', '不必', '不怎麼', '不怕', '不惟', '不成', '不拘', '不擇手段', '不料', '不日', '不時', '不是', '不曾', '不止', '不止一次', '不比', '不消', '不滿', '不然', '不然的話', '不特', '不獨', '不由得', '不知不覺', '不管', '不管怎樣', '不經意', '不勝', '不能', '不能不', '不至於', '不若', '不要', '不論', '不起', '不過', '不迭', '不問', '不限', '與', '與其', '與其說', '與否', '與此同時', '且', '且不說', '且說', '兩者', '個', '個人', '個別', '臨', '臨到', '為', '為了', '為什麼', '為何', '為止', '為此', '為著', '舉凡', '乃', '乃至', '乃至於', '麼', '之', '之一', '之所以', '之類', '烏乎', '乎', '乘', '乘勢', '乘機', '乘虛', '乘隙', '也', '也好', '也就是說', '也罷', '了', '二來', '二話不說', '二話沒說', '於', '於是', '於是乎', '云云', '云爾', '互相', '些', '交口', '亦', '親口', '親手', '親眼', '親自', '親身', '人', '人人', '人們', '人家', '什麼', '什麼樣', '今', '介於', '仍', '仍舊', '仍然', '從', '從不', '從嚴', '從中', '從今以後', '從優', '從古到今', '從古至今', '從頭', '從寬', '從小', '從新', '從無到有', '從早到晚', '從未', '從來', '從此', '從此以後', '從而', '從輕', '從速', '從重', '他', '他人', '他們', '他是', '以', '以上', '以為', '以便', '以免', '以及', '以故', '以期', '以來', '以至', '以至於', '以致', '們', '任', '任何', '任憑', '夥同', '會', '傳說', '傳聞', '似的', '但', '但凡', '但願', '但是', '何', '何樂而不為', '何以', '何況', '何處', '何妨', '何嘗', '何必', '何時', '何止', '何苦', '何須', '餘外', '作為', '你', '你們', '你是', '使', '使得', '例如', '依', '依據', '依照', '便於', '俺', '俺們', '倍加', '倍感', '倒不如', '倒不如說', '倒是', '倘', '倘使', '倘或', '倘然', '倘若', '借', '藉以', '藉此', '假使', '假如', '假若', '偏偏', '偶爾', '偶而', '儻然', '像', '兒', '元／噸', '充其極', '充其量', '充分', '先不先', '光是', '全體', '全力', '全年', '全然', '全身心', '全部', '全都', '八成', '公然', '兮', '共總', '關於', '其', '其一', '其中', '其二', '其他', '其餘', '其後', '其它', '其實', '其次', '具體地說', '具體來說', '具體說來', '兼之', '內', '再', '再其次', '再則', '再有', '再次', '再者', '再者說', '再說', '冒', '衝', '決不', '決非', '況且', '湊巧', '凝神', '幾', '幾乎', '幾度', '幾時', '幾番', '幾經', '凡', '凡是', '憑', '憑藉', '出於', '出去', '出來', '分別', '分頭', '分期分批', '切不可', '切切', '切勿', '切莫', '則', '則甚', '剛好', '剛巧', '剛才', '別', '別人', '別處', '別是', '別的', '別管', '別說', '到', '到了兒', '到處', '到頭', '到頭來', '到底', '到目前為止', '前後', '前此', '前者', '加上', '加之', '加以', '動不動', '動輒', '勃然', '匆匆', '千萬千萬', '單單', '單純', '即', '即令', '即使', '即便', '即刻', '即如', '即將', '即或', '即是說', '即若', '卻', '去', '又', '又及', '及', '及其', '及至', '反之', '反之亦然', '反之則', '反倒', '反倒是', '反手', '反而', '反過來', '反過來說', '取道', '受到', '另', '另一個', '另一方面', '另外', '另悉', '另方面', '另行', '只', '只當', '只怕', '只是', '只有', '只消', '只要', '只限', '叫', '叮咚', '可', '可以', '可好', '可是', '可能', '可見', '各', '各個', '各位', '各式', '各種', '各自', '同', '同時', '後', '後來', '後者', '向', '向使', '向著', '嚇', '嗎', '否則', '吧', '吧噠', '吱', '呀', '呃', '呆呆地', '嘔', '唄', '嗚', '嗚呼', '呢', '呵', '呵呵', '呸', '呼哧', '呼啦', '咋', '和', '咚', '咦', '咧', '咱', '咱們', '咳', '哇', '哈', '哈哈', '哉', '哎', '哎呀', '哎喲', '嘩', '嘩啦', '喲', '哦', '哩', '哪', '哪個', '哪些', '哪兒', '哪天', '哪年', '哪怕', '哪樣', '哪邊', '哪裡', '哼', '哼唷', '唉', '唯有', '啊', '啊呀', '啊哈', '啊喲', '啐', '啥', '啦', '啪達', '啷噹', '餵', '喏', '喔唷', '嘍', '嗡', '嗡嗡', '嗬', '嗯', '噯', '嘎', '嘎嘎', '嘎登', '噓', '嘛', '嘻', '嘿', '嘿嘿', '因', '因為', '因了', '因此', '因著', '因而', '固然', '在', '在下', '在於', '地', '基於', '基本', '基本上', '處在', '處處', '多', '多麼', '多虧', '多多', '多多少少', '多多益善', '多少', '多年前', '多年來', '多次', '夠瞧的', '大', '大不了', '大舉', '大體上', '大凡', '大多', '大大', '大家', '大張旗鼓', '大抵', '大概', '大略', '大約', '大致', '大都', '大面兒上', '奮勇', '她', '她們', '她是', '好', '好在', '如', '如上', '如上所述', '如下', '如今', '如何', '如其', '如前所述', '如同', '如常', '如是', '如期', '如果', '如次', '如此', '如此等等', '如若', '始而', '姑且', '存心', '孰料', '孰知', '寧', '寧可', '寧願', '寧肯', '它', '它們', '它是', '對', '對於', '對待', '對方', '對比', '將', '將才', '將要', '將近', '小', '爾', '爾後', '爾爾', '爾等', '尚且', '就', '就地', '就是', '就是了', '就是說', '就此', '就算', '就要', '盡', '盡可能', '盡如人意', '盡心盡力', '盡心竭力', '盡快', '儘早', '盡然', '儘管', '儘管如此', '盡量', '局外', '居然', '屆時', '屢屢', '屢次', '屢次三番', '豈但', '豈止', '豈非', '川流不息', '差一點', '差不多', '己', '已', '已矣', '巴', '巴巴', '常言說', '常言說得好', '常言道', '平素', '年復一年', '並', '並且', '並排', '並無', '並沒', '並沒有', '並肩', '並非', '庶乎', '庶幾', '開外', '開始', '彈指之間', '歸', '歸根到底', '歸根結底', '歸齊', '當', '當下', '當中', '當兒', '當即', '當口兒', '當地', '當場', '當頭', '當庭', '當然', '當真', '當著', '徹夜', '彼', '彼時', '彼此', '往', '待', '待到', '很', '很多', '很少', '得', '得了', '得天獨厚', '得起', '必定', '必將', '必須', '快要', '忽地', '忽然', '怎', '怎麼', '怎麼辦', '怎麼樣', '怎奈', '怎樣', '急匆匆', '怪不得', '總之', '總的來看', '總的來說', '總的說來', '總而言之', '恍然', '恐怕', '恰似', '恰好', '恰如', '恰巧', '恰恰', '恰恰相反', '恰逢', '您', '您們', '您是', '惟其', '慣常', '憤然', '慢說', '成年累月', '成心', '我', '我們', '我是', '或', '或則', '或多或少', '或是', '或曰', '或者', '或許', '截然', '截至', '所', '所以', '所在', '所幸', '所有', '才', '才能', '撲通', '打', '打從', '打開天窗說亮話', '把', '抑或', '抽冷子', '攔腰', '拿', '按', '按時', '按期', '按照', '按理', '按說', '挨個', '挨家挨戶', '挨次', '挨著', '挨門挨戶', '挨門逐戶', '換句話說', '換言之', '據', '據實', '據悉', '據我所知', '據此', '據稱', '據說', '接下來', '接著', '接連不斷', '故', '故意', '故此', '故而', '敞開兒', '敢於', '敢情', '數/', '斷然', '方才', '方能', '旁人', '無', '無寧', '無論', '既', '既往', '既是', '既然', '日復一日', '日漸', '日益', '日臻', '日見', '時候', '昂然', '是', '是以', '是的', '暗中', '暗地裡', '暗自', '更為', '更加', '更進一步', '曾', '替', '替代', '最', '最後', '有', '有些', '有關', '有及', '有時', '有的', '有的是', '望', '朝', '朝著', '末##末', '本', '本人', '本地', '本著', '本身', '權時', '來', '來不及', '來得及', '來看', '來著', '來自', '來講', '來說', '極為', '極了', '極其', '極力', '極大', '極度', '極端', '果然', '果真', '某', '某個', '某些', '某某', '根據', '格外', '次第', '歟', '正值', '正如', '正巧', '正是', '此', '此中', '此後', '此地', '此處', '此外', '此時', '此次', '此間', '毋寧', '每', '每當', '每時每刻', '每每', '每逢', '比', '比及', '比如', '比如說', '比方', '比照', '比起', '畢竟', '毫不', '毫無', '毫無例外', '毫無保留地', '沙沙', '沒奈何', '沒有', '沿', '沿著', '漫說', '焉', '然則', '然後', '然而', '照', '照著', '牢牢', '猶且', '猶自', '獨自', '猛然', '猛然間', '率爾', '率然', '理應', '理當', '理該', '瑟瑟', '甚且', '甚麼', '甚或', '甚而', '甚至', '甚至於', '用', '用來', '甭', '由', '由於', '由是', '由此', '由此可見', '略為', '略加', '略微', '的', '的確', '的話', '皆可', '直到', '相對而言', '省得', '看', '看上去', '看來', '看樣子', '看起來', '眨眼', '著', '著呢', '矣', '矣乎', '矣哉', '砰', '碰巧', '離', '種', '究竟', '窮年累月', '立刻', '立地', '立時', '立馬', '竟然', '竟而', '第', '第二', '等', '等到', '等等', '策略地', '簡直', '簡而言之', '簡言之', '管', '類如', '精光', '緊接著', '累年', '累次', '純粹', '縱', '縱令', '縱使', '縱然', '經', '經常', '經過', '結果', '給', '絕不', '絕對', '絕非', '絕頂', '繼之', '繼後', '繼而', '綜上所述', '縷縷', '罷了', '老是', '老老實實', '者', '而', '而且', '而況', '而又', '而後', '而外', '而已', '而是', '而言', '而論', '聯袂', '背地裡', '背靠背', '能', '能否', '騰', '自', '自個兒', '自從', '自各兒', '自後', '自家', '自己', '自打', '自身', '至', '至於', '至今', '至若', '致', '般的', '若', '若夫', '若是', '若果', '若非', '莫不', '莫不然', '莫如', '莫若', '莫非', '藉以', '雖', '雖則', '雖然', '雖說', '被', '要', '要不', '要不是', '要不然', '要么', '要是', '譬喻', '譬如', '讓', '許多', '論', '論說', '設使', '設或', '設若', '誠如', '誠然', '話說', '該', '該當', '說來', '請勿', '諸', '諸位', '諸如', '誰', '誰人', '誰料', '誰知', '豁然', '賊死', '賴以', '趕', '趕快', '趕早不趕晚', '起', '起先', '起初', '起頭', '起來', '起見', '起首', '趁', '趁便', '趁勢', '趁早', '趁機', '趁熱', '趁著', '越是', '距', '跟', '路經', '轟然', '較', '較為', '較之', '較比', '邊', '達旦', '過', '過於', '近幾年來', '近年來', '近來', '還', '還是', '還有', '還要', '這', '這一來', '這個', '這麼', '這麼些', '這麼樣', '這麼點兒', '這些', '這會兒', '這兒', '這就是說', '這時', '這樣', '這次', '這般', '這邊', '這裡', '進去', '進來', '進而', '連', '連同', '連聲', '連日', '連日來', '連袂', '連連', '遲早', '迫於', '逐步', '通過', '遵循', '遵照', '那', '那個', '那麼', '那麼些', '那麼樣', '那些', '那會兒', '那兒', '那時', '那末', '那樣', '那般', '那邊', '那裡', '都', '鄙人', '鑑於', '針對', '長期以來', '長此下去', '長話短說', '間或', '阿', '陡然', '除', '除了', '除卻', '除去', '除外', '除開', '除此', '除此之外', '除此以外', '除此而外', '除非', '隨', '隨後', '隨時', '隨著', '隔夜', '隔日', '難得', '難怪', '難說', '難道', '難道說', '非但', '非常', '非徒', '非得', '非特', '非獨', '靠', '頂多', '頃刻', '頃刻之間', '頃刻間', '順', '順著', '頓時', '風雨無阻', '首先', '馬上', '高低', '默然', '默默地', '！', '＃', '％', '＆', '＇', '（', '）', '）÷（１－', '）、', '＊', '＋', '＋ξ', '＋＋', '，', '，也', '－', '－β', '－－', '－［＊］－', '．', '／', '０：２', '１．', '１２％', '２．３％', '５：０', '：', '；', '＜', '＜±', '＜Δ', '＜λ', '＜φ', '＜＜', '＝', '＝″', '＝☆', '＝（', '＝－', '＝［', '＝｛', '＞', '＞λ', '？', 'Ａ', 'ＬＩ', 'Ｒ．Ｌ．', 'ＺＸＦＩＴＬ', '［', '［①①］', '［①②］', '［①③］', '［①④］', '［①⑤］', '［①⑥］', '［①⑦］', '［①⑧］', '［①⑨］', '［①Ａ］', '［①Ｂ］', '［①Ｃ］', '［①Ｄ］', '［①Ｅ］', '［①］', '［①ａ］', '［①ｃ］', '［①ｄ］', '［①ｅ］', '［①ｆ］', '［①ｇ］', '［①ｈ］', '［①ｉ］', '［①ｏ］', '［②', '［②①］', '［②②］', '［②③］', '［②④', '［②⑤］', '［②⑥］', '［②⑦］', '［②⑧］', '［②⑩］', '［②Ｂ］', '［②Ｇ］', '［②］', '［②ａ］', '［②ｂ］', '［②ｃ］', '［②ｄ］', '［②ｅ］', '［②ｆ］', '［②ｇ］', '［②ｈ］', '［②ｉ］', '［②ｊ］', '［③①］', '［③⑩］', '［③Ｆ］', '［③］', '［③ａ］', '［③ｂ］', '［③ｃ］', '［③ｄ］', '［③ｅ］', '［③ｇ］', '［③ｈ］', '［④］', '［④ａ］', '［④ｂ］', '［④ｃ］', '［④ｄ］', '［④ｅ］', '［⑤］', '［⑤］］', '［⑤ａ］', '［⑤ｂ］', '［⑤ｄ］', '［⑤ｅ］', '［⑤ｆ］', '［⑥］', '［⑦］', '［⑧］', '［⑨］', '［⑩］', '［＊］', '［－', '［］', '］', '］∧′＝［', '］［', '＿', 'ａ］', 'ｂ］', 'ｃ］', 'ｅ］', 'ｆ］', 'ｎｇ昉', '｛－', '｝', '｝＞', '～', '～±', '～＋']
        all_stopwords = np.hstack((stopwords,myself_stopword))
        df["cut_title"] = df["clean_title"].apply(lambda x: ",".join([w for w in list(jb.cut(x)) if w not in all_stopwords]))
        return df

    # 建立停用詞字典
    def word_dict(df):
        x = df["cut_title"].values
        # 總詞彙矩陣
        all_word = []
        for i in range(len(df)):
            i_word = x[i].split(",")
            all_word = np.hstack((all_word,i_word))
        # 詞彙字典與頻率
        dic={}
        for word in all_word:
            if word not in dic:
                dic[word] = 1
            else:
                dic[word] = dic[word] + 1
        return dic
    
    df["clean_title"] = df["title"].apply(remove_punctuation)
    df = delete_stopwords(df,myself_stopword)
    keyword = pd.DataFrame(list(word_dict(df).items()), columns=['Keyword', 'Frequency'])
    keyword.index = keyword['Frequency']
    keyword = keyword.sort_index(ascending=False)
    keyword_df = keyword.reset_index(drop=True)
    return keyword_df

def main(request):
    # *** 選擇從哪天開始爬取文章 ***
    def week_date_range(datetime_date):
      # 為期一週
        date_range = []
        for i in range(7):
            date = (datetime_date - datetime.timedelta(days=i)).strftime('%m/%d').lstrip('0')
            date_range.append(date)
        return date_range

    def get_ppt_page(url):
        resp = requests.get(url=url)
        if resp.status_code != 200:
            return None
        else:
            return resp.text
    
    def get_pageinfo(resdata,domain_url):
        soup = BeautifulSoup(resdata, 'html5lib')
        # 取得上一頁按鈕
        paging_div = soup.find('div', 'btn-group btn-group-paging')
        prev_url = paging_div.find_all('a')[1]['href']
        
        # 儲存取得的文章資料<div class="r-ent"></div>
        pptdata = []  
        date_divs = soup.find_all('div', 'r-ent')
        
        num = 0
        whether_prev = True
        for k in date_divs:
            # 判斷文章是否在一週內發佈
            week_date = k.find('div', 'date').text.strip() in date_range
            
            # 代表最早的日期，就不符合條件，所以不用下一頁
            if num == 0:
                if week_date == False:
                    whether_prev = False
            # 一周內發布       
            if week_date:
                # 發布日期
                post_date = k.find('div', 'date').text.strip()
                #print("發布日期",post_date)
                
                # 推文數
                push_count = k.find('div', 'nrec').text
                push_num = 0
                if push_count:
                    try:
                        push_num = int(push_count)  
                    except ValueError:
                        # 若轉換失敗，可能是'爆'或 'X1', 'X2'
                        if push_count == '爆':
                            push_num = 100
                        elif push_count.startswith('X'):
                            push_num = -100
                        else:
                            push_num = 0
                #print("推文數",push_num)
                
                # 有超連結，代表文章存在
                if k.find('a'): 
                    # 文章標題
                    title = k.find('a').text
                    #print("標題",title)
                    # 文章連結
                    href = k.find('a')['href']
                    #print("標題連結",domain_url+href)
                    
                    pptdata.append({
                        'date': post_date,
                        'push_num': push_num,
                        'title': title,
                        'href': domain_url+href
                    })
            
            #每做完一次，num+1
            num = num + 1 
        df = pd.DataFrame(pptdata, columns=['date','push_num', 'title', 'href'])
        
        return prev_url,df,whether_prev    

    # 排序：推廣數 #推廣數：'push_num'; 日期：'date'
    sort_index = 'date'
    # 為期一週的日期
    datetime_date = datetime.date.today()
    date_range = week_date_range(datetime_date)
    # *** 選擇保險版 ***
    domain_url = 'https://www.ptt.cc'
    Insurance = '/bbs/Insurance/index.html'
    # 第一頁
    web_url = domain_url + Insurance
    ppt_page = get_ppt_page(web_url)
    prev_href,df,whether_prev = get_pageinfo(ppt_page,domain_url) 
    
    # 往前一頁
    while whether_prev:
        web_url = domain_url+prev_href
        ppt_page = get_ppt_page(web_url)
        prev_href,df_next,whether_prev = get_pageinfo(ppt_page,domain_url) 
        df = pd.concat([df,df_next])
    
    df.index = df[sort_index]
    df = df.sort_index(ascending=False)
    df = df.reset_index(drop=True)

    df.to_csv("/tmp/ptt_crawler.csv", index=False)
    upload_blob("ptt-crawler-cathay", "/tmp/ptt_crawler.csv", "ptt_crawler.csv")
    

    ptt_kerword_df = kerword(df)
    ptt_kerword_df.to_csv("/tmp/ptt_kerword.csv", index=False)
    upload_blob("ptt-crawler-cathay", "/tmp/ptt_kerword.csv", "ptt_kerword.csv")
